In [7]:
import os
import sys
sys.path.append('../code')

In [8]:
import requests
import time
import os
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import json

# Import Google Generative AI SDK and tool schema components
import google.generativeai as genai
from google.generativeai.types import FunctionDeclaration, Tool

# Load Gemini API key from environment and configure model
genai.configure(api_key=os.getenv("GEMINI_API_KEY"))
model = genai.GenerativeModel("gemini-2.0-flash")


from llm_connect import get_response
from context import get_all_candidates

# Interview Scheduler

In [9]:
# Load Calendly API key from environment
load_dotenv()
CALENDLY_API_KEY = os.getenv('CALENDLY_API_KEY')

# Set up headers for authenticated Calendly API requests
headers = {
    'Authorization': f"Bearer {CALENDLY_API_KEY}",
    'Content-Type': 'application/json'
}

In [10]:
# Get the authenticated user's Calendly URI
def get_user_uri():
    url = 'https://api.calendly.com/users/me'
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    return response.json()['resource']['uri']

# Retrieve the scheduling link for the first event type
def get_event_type_link(user_uri):
    url = f"https://api.calendly.com/event_types?user={user_uri}"
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    return response.json()['collection'][0]['scheduling_url']

# Generate a scheduling URL for the candidate with pre-filled name and email
def schedule_interview(name: str, email: str) -> str:
    user_uri = get_user_uri()
    event_url = get_event_type_link(user_uri)
    return f"{event_url}?name={name.replace(' ', '+')}&email={email}"

In [11]:
# Define a function tool schema for Gemini to use when scheduling interviews
schedule_params = {
    "type": "object",
    "properties": {
        "name": {"type": "string", "description": "Candidate's full name"},
        "email": {"type": "string", "description": "Candidate's email"}
    },
    "required": ["name", "email"]
}

schedule_func = FunctionDeclaration(
    name="schedule_interview",
    description="Schedule an interview for a candidate by name and email.",
    parameters=schedule_params
)

# Register the tool so Gemini can call it
schedule_tool = Tool(function_declarations=[schedule_func])

In [14]:
model = genai.GenerativeModel(
    model_name="gemini-2.0-flash",
    tools=[schedule_tool]
)

# Start a new Gemini conversation session
chat = model.start_chat()

# Filter candidates by job and score threshold
with open("data/mcp_context.json", "r") as f:
        mcp_data = json.load(f)
    
     #Job Data
job_id = next(iter(mcp_data['jobs']))


candidates_dict = get_all_candidates()

candidates = [
    {
        'candidate_id': c['Candidate ID'],
        'name': c['Name'],
        'email': c['Email'],
        'score': c.get('avg_score', 0)
    }
    for c in candidates_dict.values()
    if c.get('job_id') == job_id and c.get('avg_score', 0) >= 6.5
]

# Ask Gemini to use the tool and schedule all qualified candidates
prompt = (
    'You are a recruiting assistant. Here is a list of candidates: \n\n'
    f"{json.dumps(candidates, indent=2)}\n\n"
    'Schedule interviews using the `schedule_interview` tool for all candidates.'
    'Return nothing, invoke the tool'
)

# Call Gemini with tool available
response = chat.send_message(prompt)

# Handle LLM responses and extract function calls
scheduled = []

for part in response.candidates[0].content.parts:
    if part.function_call:
        fn = part.function_call.name
        args = part.function_call.args
        if fn == "schedule_interview":
            try:
                link = schedule_interview(**args)
                scheduled.append({
                    "Name": args["name"],
                    "Email": args["email"],
                    "Interview Link": link
                })
                print(f"✅ Scheduled: {args['name']} → {link}")
            except Exception as e:
                print(f"❌ Failed to schedule {args['name']}: {e}")


out_df = pd.DataFrame(scheduled)
out_df.to_csv("data/scheduled_candidates.csv", index=False)
print("\n📁 Output saved to data/scheduled_candidates.csv")

✅ Scheduled: Alexander Prince → https://calendly.com/roguchi-ucsd/30min?name=Alexander+Prince&email=alexanderprince@example.com
✅ Scheduled: Dawn Osborn → https://calendly.com/roguchi-ucsd/30min?name=Dawn+Osborn&email=dawnosborn@example.com
✅ Scheduled: Cindy Lambert → https://calendly.com/roguchi-ucsd/30min?name=Cindy+Lambert&email=cindylambert@example.com
✅ Scheduled: Nathan Rodriguez → https://calendly.com/roguchi-ucsd/30min?name=Nathan+Rodriguez&email=nathanrodriguez@example.com
✅ Scheduled: Lauren Powers → https://calendly.com/roguchi-ucsd/30min?name=Lauren+Powers&email=laurenpowers@example.com
✅ Scheduled: Marissa Drake → https://calendly.com/roguchi-ucsd/30min?name=Marissa+Drake&email=marissadrake@example.com
✅ Scheduled: Melissa Brady → https://calendly.com/roguchi-ucsd/30min?name=Melissa+Brady&email=melissabrady@example.com
✅ Scheduled: David Sloan → https://calendly.com/roguchi-ucsd/30min?name=David+Sloan&email=davidsloan@example.com
✅ Scheduled: Brandon Martin → https://cale

In [15]:
# Load and display the saved list of scheduled candidates
scheduled_candidates = pd.read_csv('data/scheduled_candidates.csv')
scheduled_candidates

,Name,Email,Interview Link
0,Alexander Prince,alexanderprince@example.com,https://calendly.com/roguchi-ucsd/30min?name=A...
1,Dawn Osborn,dawnosborn@example.com,https://calendly.com/roguchi-ucsd/30min?name=D...
2,Cindy Lambert,cindylambert@example.com,https://calendly.com/roguchi-ucsd/30min?name=C...
3,Nathan Rodriguez,nathanrodriguez@example.com,https://calendly.com/roguchi-ucsd/30min?name=N...
4,Lauren Powers,laurenpowers@example.com,https://calendly.com/roguchi-ucsd/30min?name=L...
5,Marissa Drake,marissadrake@example.com,https://calendly.com/roguchi-ucsd/30min?name=M...
6,Melissa Brady,melissabrady@example.com,https://calendly.com/roguchi-ucsd/30min?name=M...
7,David Sloan,davidsloan@example.com,https://calendly.com/roguchi-ucsd/30min?name=D...
8,Brandon Martin,brandonmartin@example.com,https://calendly.com/roguchi-ucsd/30min?name=B...


## Interview Scheduler: Automating Candidate Outreach and Scheduling with GenAI

This notebook implements an automated workflow to identify top candidates and schedule interviews using a combination of large language models (LLMs) and the Calendly API. It acts as a decision-support layer between candidate evaluation and actual interview logistics. Moreover, this section is to implement the tool calling aspect that LLMs can utilize. Out of the two models that are in use in this workflow, we use Gemini as it the ability to readily access custom tools and APIs.

### Objectives
- Filter and prioritize candidates from the existing context (`mcp_context.json`)
- Use a Gemini-powered LLM to reason about interview suitability and scheduling logic
- Automate booking via Calendly's scheduling API

### Key Components

1. **Context Access**:
   - Loads structured candidate profiles from the shared context store (`get_all_candidates()`).
   - Only candidates who meet specific criteria (e.g., high scores, relevant skills) are considered for interview scheduling.

2. **LLM Tool Calling with Gemini**:
   - Defines a function schema that lets the Gemini model call a scheduling function with selected parameters.
   - Uses Gemini to determine who to interview and when based on candidate metadata.

3. **Calendly API Integration**:
   - Authenticates using a secure API key and sends POST requests to book interviews.
   - Each API request includes candidate name, selected time, and event details.

4. **Automation and Flexibility**:
   - The system could easily be extended to include email notifications, interviewer matching, or calendar syncing.
   - The use of Gemini enables flexible, adaptive reasoning over candidate features.

### Outputs
- Confirmation logs for successful interview bookings
- Full traceability from candidate evaluation to scheduling
- Readiness for integration into a broader GenAI-augmented hiring workflow

This notebook demonstrates how GenAI can bridge the gap between candidate intelligence and real-world actions like interview scheduling, streamlining a typically manual and fragmented HR process. Instead of allowing recruiters to manually send out emails to qualified candidates, this can save a noticeable amount of time by allowing AI to do this process itself.
